<a href="https://colab.research.google.com/github/Yi-Wei-Lin/Tibame_ML_20210602/blob/main/%E9%A6%99%E8%95%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
train_df = pd.read_csv("test.csv", encoding="utf-8")
# x_train.insert(loc = 8, column="month",value=2)

# Date 民國 to 西元
def date_df(d):
  d = d.split("/")
  year = str(int(d[0])+1911)
  month = (d[1])
  day = (d[2])
  d = f"{year}/{month}/{day}"
  return d
date_df = pd.DataFrame({"Date":train_df["Date"].apply(date_df)})
train_df.update(date_df)

# Month method
def Month(m):
  Month_data = int(m.split("/")[1])
  return Month_data
train_df["Month"] = train_df["Date"].apply(Month)

# Week_day method
def week_day(d):
  d = d.split("/")
  year = int(d[0])
  month = int(d[1])
  day = int(d[2])
  temp = pd.Timestamp(f"{year}-{month}-{day}")
  week_day = temp.dayofweek + 1 #temp.dayofweek 0(禮拜一) 開始, +1是因為比較直接禮拜幾
  return week_day
train_df["Week_day"] = train_df["Date"].apply(week_day)
train_df["Year"] = train_df["Date"].apply(lambda d:d.split("/")[0])

# Rest_market
market_df = pd.DataFrame({"Market":train_df["Market"].apply(lambda t:t.split(" ")[1])})
train_df.update(market_df)

train_df

,Date,Market,Product,Up_price,Mid_price,Low_price,Avg_price,Volume,Month,Week_day,Year
0,2001/01/01,台北二,A1 香蕉,35,30.0,26.7,30.4,"2,679",1,1,2001
1,2001/01/02,台北二,A1 香蕉,32.9,25.5,22.0,26.3,"4,589",1,2,2001
2,2001/01/03,台北二,A1 香蕉,32.7,26.4,22.0,26.8,"5,437",1,3,2001
3,2001/01/05,台北二,A1 香蕉,32.5,24.6,19.7,25.2,"7,835",1,5,2001
4,2001/01/06,台北二,A1 香蕉,29.8,24.1,19.2,24.3,"5,896",1,6,2001
...,...,...,...,...,...,...,...,...,...,...,...
56452,2021/06/12,台東市,A1 香蕉,32,21.5,15.0,22.8,"3,007",6,6,2021
56453,2021/06/13,台東市,A1 香蕉,30,26.2,18.0,27.9,"3,558",6,7,2021
56454,2021/06/14,台東市,A1 香蕉,30,26.2,18.0,22.9,"2,857",6,1,2021
56455,2021/06/17,台東市,A1 香蕉,33,22.9,15.0,20.8,"3,052",6,4,2021


In [21]:
t_index = pd.date_range('2001-01-01', '2021-06-19', freq='D')
# train_df
print(t_index)
T_df = pd.DataFrame(t_index, columns=['Date'])
train_df.merge(T_df, how='right', sort=True)
train_df

DatetimeIndex(['2001-01-01', '2001-01-02', '2001-01-03', '2001-01-04',
               '2001-01-05', '2001-01-06', '2001-01-07', '2001-01-08',
               '2001-01-09', '2001-01-10',
               ...
               '2021-06-10', '2021-06-11', '2021-06-12', '2021-06-13',
               '2021-06-14', '2021-06-15', '2021-06-16', '2021-06-17',
               '2021-06-18', '2021-06-19'],
              dtype='datetime64[ns]', length=7475, freq='D')


ValueError: ignored

In [ ]:
老師不好意思, 想詢問一下專題上遇到的問題, 第一張圖是專題大概的簡介(農產品價格預測), 第二章圖是目前遇到的困難, 
由於目前 input data 有很多批發市場(~13個左右), 每個市場都有不同的價格, 之前聽宇春老師的意思好像可以全部 data
都丟進去 model 裡面 training, 可是每種批發價格都不一樣, 這樣預測的價格不會有問題嗎, 還是我有 13 個批發市場
我可以 output 13個批發市場的預測價格呢??, 我有點不太懂, 所以想詢問一下你的建議, 感謝.
另外想問一下 13個批發市場每天不一定都有開市, 有些市場與其他市場休市時間不一樣, 假設110/06/06 有13個開市13個價格
跟 110/06/07 有10個開市10個價格, 這樣 input data 會不會有問題, 是否要補齊其他3個休市的缺失值呢??
然後氣候特徵工程的補缺失值得部分, 像是我有一些氣象資料有缺可能某些縣市會缺很多 data, 這種是要靠鄰近的縣市補強, 還是就直接省去呢??